# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")
print("PRICE_DATA is:", PRICE_DATA)
stock_files = glob(os.path.join(PRICE_DATA,"*","*","*.parquet"), recursive = True)

PRICE_DATA is: ../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Read all parquet files
dd_prices = dd.read_parquet(stock_files)

# Add lags for Close and Adj_Close
dd_prices = dd_prices.assign(
    Close_lag_1 = dd_prices["Close"].shift(1),
    Adj_Close_lag_1 = dd_prices["Adj Close"].shift(1)
)

# Returns based on Close
dd_prices = dd_prices.assign(
    returns = (dd_prices["Close"] / dd_prices["Close_lag_1"]) - 1
)

# Day's High minus Low
dd_prices = dd_prices.assign(
    hi_lo_range = dd_prices["High"] - dd_prices["Low"]
)

# Assign the result
dd_feat = dd_prices


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.

# 1. Compute the full Dask DataFrame into pandas
fDD = dd_feat.compute()

# 2. Sort by ticker and Date
fDD = fDD.sort_values(["ticker", "Date"])

# 3. Window of 10 days
fDD["ma10_returns"] = fDD.groupby("ticker")["returns"].transform(lambda x: x.rolling(10).mean())

print(fDD.head(30))


             Date   Open   High    Low  Close  Adj Close      Volume   source  \
137315 2001-07-19  15.10  15.29  15.00  15.17  11.404394  34994300.0  ACN.csv   
137316 2001-07-20  15.05  15.05  14.80  15.01  11.284108   9238500.0  ACN.csv   
137317 2001-07-23  15.00  15.01  14.55  15.00  11.276587   7501000.0  ACN.csv   
137318 2001-07-24  14.95  14.97  14.70  14.86  11.171341   3537300.0  ACN.csv   
137319 2001-07-25  14.70  14.95  14.65  14.95  11.238999   4208100.0  ACN.csv   
137320 2001-07-26  14.95  14.99  14.50  14.50  10.900705   6335300.0  ACN.csv   
137321 2001-07-27  14.51  14.59  14.50  14.51  10.908223   3524000.0  ACN.csv   
137322 2001-07-30  14.50  14.78  14.50  14.70  11.051059   3654300.0  ACN.csv   
137323 2001-07-31  14.71  15.01  14.60  14.96  11.246520   1429000.0  ACN.csv   
137324 2001-08-01  15.00  15.50  14.90  15.50  11.652478   2087900.0  ACN.csv   
137325 2001-08-02  15.40  15.50  15.10  15.40  11.577302   1717400.0  ACN.csv   
137326 2001-08-03  15.40  15

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Answer: It depends on the data size. If the data size is small, it is better to convert to pandas since it is simpler. But if the data is large and can not put directly into memory, we can use Dask. I heard that using Dask is trickier than using Pandas.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.